In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import logging
from datetime import datetime, timedelta
import time
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("multi_year_momentum.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

def fetch_stock_data(symbol, exchange_suffix, end_date, max_retries=3, fallback_exchange=None):
    """
    Fetch historical stock data with retry mechanism and exchange fallback
    
    This function now fetches max available data to ensure we have enough history
    """
    retries = 0
    ticker_symbol = f"{symbol}{exchange_suffix}"
    
    # Set start_date to be far back to get as much data as available
    start_date = "2015-01-01"  # This ensures we have enough historical data
    
    while retries < max_retries:
        try:
            data = yf.download(ticker_symbol, start=start_date, end=end_date, progress=False)
            
            # Check if we got enough data
            if len(data) > 250:  # Require at least a year of trading data (approx 250 days)
                logger.info(f"Successfully fetched data for {ticker_symbol}: {len(data)} days")
                return data, ticker_symbol
            else:
                logger.warning(f"Insufficient data for {ticker_symbol}. Got only {len(data)} rows.")
                
        except Exception as e:
            logger.warning(f"Error fetching {ticker_symbol}: {str(e)}")
        
        retries += 1
        time.sleep(1)  # Wait before retry
    
    # If primary exchange failed and fallback is provided, try fallback
    if fallback_exchange and exchange_suffix != fallback_exchange:
        logger.info(f"Trying fallback exchange for {symbol} with {fallback_exchange}")
        try:
            fallback_symbol = f"{symbol}{fallback_exchange}"
            data = yf.download(fallback_symbol, start=start_date, end=end_date, progress=False)
            
            if len(data) > 250:
                logger.info(f"Successfully fetched data from fallback exchange for {fallback_symbol}: {len(data)} days")
                return data, fallback_symbol
        except Exception as e:
            logger.warning(f"Fallback exchange also failed for {symbol}: {str(e)}")
    
    logger.error(f"Failed to fetch data for {symbol} after all attempts")
    return None, ticker_symbol

def get_close_prices(stock_data):
    """
    Extract the closing prices from the stock data, handling MultiIndex
    """
    try:
        # Check if we have a MultiIndex DataFrame
        if isinstance(stock_data.columns, pd.MultiIndex):
            logger.info("Detected MultiIndex DataFrame")
            
            # Check for 'Close' in the first level of MultiIndex
            if 'Close' in stock_data.columns.get_level_values(0):
                logger.info("Using 'Close' prices from MultiIndex")
                close_prices = stock_data['Close']
                
                # Important fix: If close_prices is a DataFrame, get the first column
                if isinstance(close_prices, pd.DataFrame):
                    logger.info(f"Close prices has multiple columns: {close_prices.columns}")
                    close_prices = close_prices.iloc[:, 0]  # Get the first column
                    logger.info("Using first column of Close prices")
            else:
                # If no 'Close', check what's available in first level
                available_cols = stock_data.columns.get_level_values(0).unique().tolist()
                logger.info(f"Available first-level columns: {available_cols}")
                
                # Try to find a suitable price column
                price_col = None
                for col in available_cols:
                    if isinstance(col, str) and col.lower() in ['close', 'adj close', 'adjclose', 'last', 'price']:
                        price_col = col
                        break
                
                if price_col:
                    logger.info(f"Using '{price_col}' as price column from MultiIndex")
                    close_prices = stock_data[price_col]
                    
                    # If close_prices is still a DataFrame, get the first column
                    if isinstance(close_prices, pd.DataFrame):
                        close_prices = close_prices.iloc[:, 0]
                else:
                    logger.error("No suitable price column found in the MultiIndex")
                    return None
        else:
            # For standard DataFrame
            available_columns = stock_data.columns.tolist()
            logger.info(f"Available columns: {available_columns}")
            
            # Try different column names for closing prices with fallback mechanism
            if 'Adj Close' in available_columns:
                close_prices = stock_data['Adj Close']
            elif 'Close' in available_columns:
                logger.info("Using 'Close' prices as fallback instead of 'Adj Close'")
                close_prices = stock_data['Close']
            else:
                # If none of the expected columns exist, try finding a column with price-related name
                close_col = None
                for col in available_columns:
                    if isinstance(col, str) and col.lower() in ['adj close', 'adjclose', 'close', 'last', 'price']:
                        close_col = col
                        break
                
                if close_col:
                    logger.info(f"Using '{close_col}' as price column")
                    close_prices = stock_data[close_col]
                else:
                    logger.error("No suitable price column found in the data")
                    return None
        
        # Ensure we have a Series, not a DataFrame
        if isinstance(close_prices, pd.DataFrame):
            logger.info("Converting DataFrame to Series (using first column)")
            close_prices = close_prices.iloc[:, 0]
            
        return close_prices
    except Exception as e:
        logger.error(f"Error extracting close prices: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def calculate_year_momentum(close_prices, year_end_date):
    """
    Calculate momentum for a specific year ending on year_end_date
    """
    try:
        # Find start date (1 year before end date)
        year_start_date = year_end_date - timedelta(days=365)
        
        # Filter prices for the year
        year_prices = close_prices[(close_prices.index >= year_start_date) & 
                                   (close_prices.index <= year_end_date)]
        
        # Ensure we have enough data
        if len(year_prices) < 200:  # Approximately 200 trading days in a year
            logger.warning(f"Insufficient data for year ending {year_end_date.strftime('%Y-%m-%d')}: {len(year_prices)} days")
            return None, None, None
        
        # Calculate 12-month return
        start_price = year_prices.iloc[0]
        end_price = year_prices.iloc[-1]
        
        # Debug info
        logger.info(f"Start price: {start_price} ({type(start_price)}), End price: {end_price} ({type(end_price)})")
        
        # Make sure we have scalar values, not Series
        if hasattr(start_price, 'item'):
            start_price = start_price.item()
        if hasattr(end_price, 'item'):
            end_price = end_price.item()
            
        twelve_month_return = (end_price / start_price) - 1
        
        # Calculate daily returns
        daily_returns = year_prices.pct_change().dropna()
        
        # Calculate volatility (standard deviation of daily returns)
        volatility = daily_returns.std()
        
        # Make sure volatility is a scalar value, not a Series
        if hasattr(volatility, 'item'):
            volatility = volatility.item()
            
        # Calculate risk-adjusted momentum
        if volatility > 0:
            risk_adjusted_momentum = twelve_month_return / volatility
        else:
            logger.warning(f"Volatility is zero for year ending {year_end_date.strftime('%Y-%m-%d')}")
            return None, None, None
            
        return twelve_month_return, volatility, risk_adjusted_momentum
        
    except Exception as e:
        logger.error(f"Error calculating momentum for year ending {year_end_date.strftime('%Y-%m-%d')}: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
        return None, None, None

def main():
    try:
        # Step 1: Load input file
        input_file = 'name.xlsx'
        if not os.path.exists(input_file):
            logger.error(f"Input file {input_file} not found")
            return
            
        logger.info(f"Reading input file: {input_file}")
        df = pd.read_excel(input_file)
        
        # Verify required columns exist
        required_columns = ['SYMBOL', 'NAME OF COMPANY', 'INDUSTRY']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            logger.error(f"Input file missing required columns: {missing}")
            return
        
        # Step 2: Define the four yearly end dates
        year_end_dates = [
            datetime(2021, 9, 1),
            datetime(2022, 9, 1),
            datetime(2023, 9, 1),
            datetime(2024, 9, 1)
        ]
        
        # Latest date needed for data fetching (we need data from 2020-09-01 for the first calculation)
        latest_date_needed = datetime.now()
        
        # Create output excel file with initial structure
        output_file = 'multi_year_momentum.xlsx'
        writer = pd.ExcelWriter(output_file, engine='openpyxl')
        
        # Create base results dataframe
        base_columns = ['SYMBOL', 'NAME OF COMPANY', 'INDUSTRY', 'Exchange Used']
        
        # Create year-specific sheets
        for year_date in year_end_dates:
            year_sheet = f"Year_{year_date.strftime('%Y')}"
            year_df = pd.DataFrame(columns=base_columns + ['12M Return', 'Volatility (Std Dev)', 'Risk-Adjusted Momentum', 'Rank', 'Category'])
            year_df.to_excel(writer, sheet_name=year_sheet, index=False)
        
        # Save the initial structure
        writer.close()
        
        # Step 3: Process each stock and update Excel incrementally
        for i, row in df.iterrows():
            symbol = row['SYMBOL']
            company_name = row['NAME OF COMPANY']
            industry = row['INDUSTRY']
            logger.info(f"Processing {i+1}/{len(df)}: {symbol} - {company_name}")
            
            # Try NSE first, then BSE as fallback
            stock_data, exchange_used = fetch_stock_data(
                symbol, 
                '.NS', 
                latest_date_needed, 
                fallback_exchange='.BO'
            )
            
            if stock_data is not None:
                # Extract closing prices
                close_prices = get_close_prices(stock_data)
                
                if close_prices is not None:
                    # Process each year
                    for year_date in year_end_dates:
                        year_sheet = f"Year_{year_date.strftime('%Y')}"
                        
                        # Calculate momentum for this year
                        twelve_month_return, volatility, risk_adjusted_momentum = calculate_year_momentum(close_prices, year_date)
                        
                        # Load current year data from Excel
                        with pd.ExcelFile(output_file) as xls:
                            year_df = pd.read_excel(xls, sheet_name=year_sheet)
                        
                        # Create new row
                        new_row = {
                            'SYMBOL': symbol,
                            'NAME OF COMPANY': company_name,
                            'INDUSTRY': industry,
                            'Exchange Used': exchange_used,
                            '12M Return': twelve_month_return if twelve_month_return is not None else np.nan,
                            'Volatility (Std Dev)': volatility if volatility is not None else np.nan,
                            'Risk-Adjusted Momentum': risk_adjusted_momentum if risk_adjusted_momentum is not None else np.nan
                        }
                        
                        # Append row to the dataframe
                        year_df = pd.concat([year_df, pd.DataFrame([new_row])], ignore_index=True)
                        
                        # Write back to Excel
                        with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
                            year_df.to_excel(writer, sheet_name=year_sheet, index=False)
            
            # Add a small delay to avoid rate limiting
            time.sleep(0.5)
        
        # Step 4: Final processing - calculate ranks and categories for each year
        with pd.ExcelFile(output_file) as xls:
            with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
                for year_date in year_end_dates:
                    year_sheet = f"Year_{year_date.strftime('%Y')}"
                    year_df = pd.read_excel(xls, sheet_name=year_sheet)
                    
                    # Filter valid results
                    valid_results = year_df.dropna(subset=['Risk-Adjusted Momentum']).copy()
                    
                    if len(valid_results) > 0:
                        # Calculate ranks
                        valid_results['Rank'] = valid_results['Risk-Adjusted Momentum'].rank(ascending=False)
                        
                        # Categorize stocks
                        num_stocks = len(valid_results)
                        winner_threshold = int(num_stocks * 0.3)
                        loser_threshold = int(num_stocks * 0.7)
                        
                        def categorize(rank):
                            if rank <= winner_threshold:
                                return 'Winner'
                            elif rank > loser_threshold:
                                return 'Loser'
                            else:
                                return 'Neutral'
                        
                        valid_results['Category'] = valid_results['Rank'].apply(categorize)
                        
                        # Format percentage columns
                        valid_results['12M Return'] = valid_results['12M Return'].apply(lambda x: f"{x:.2%}" if pd.notnull(x) else "")
                        valid_results['Volatility (Std Dev)'] = valid_results['Volatility (Std Dev)'].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "")
                        valid_results['Risk-Adjusted Momentum'] = valid_results['Risk-Adjusted Momentum'].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "")
                        
                        # Save back to Excel
                        valid_results.to_excel(writer, sheet_name=year_sheet, index=False)
                        
                        # Log summary for this year
                        logger.info(f"Year {year_date.strftime('%Y')} - RAM calculated for {len(valid_results)} stocks")
                        logger.info(f"  Winners: {len(valid_results[valid_results['Category'] == 'Winner'])}")
                        logger.info(f"  Neutral: {len(valid_results[valid_results['Category'] == 'Neutral'])}")
                        logger.info(f"  Losers: {len(valid_results[valid_results['Category'] == 'Loser'])}")
                    else:
                        logger.warning(f"No valid results for year {year_date.strftime('%Y')}")
        
        # Create a summary sheet
        with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
            summary_df = pd.DataFrame(columns=['SYMBOL', 'NAME OF COMPANY', 'INDUSTRY'])
            
            for symbol, name, industry in zip(df['SYMBOL'], df['NAME OF COMPANY'], df['INDUSTRY']):
                years_data = {}
                
                # Collect data from all years
                for year_date in year_end_dates:
                    year = year_date.strftime('%Y')
                    year_sheet = f"Year_{year}"
                    
                    with pd.ExcelFile(output_file) as xls:
                        year_df = pd.read_excel(xls, sheet_name=year_sheet)
                        
                    stock_data = year_df[year_df['SYMBOL'] == symbol]
                    if not stock_data.empty:
                        ram = stock_data['Risk-Adjusted Momentum'].values[0] if pd.notna(stock_data['Risk-Adjusted Momentum'].values[0]) else "N/A"
                        category = stock_data['Category'].values[0] if pd.notna(stock_data['Category'].values[0]) else "N/A"
                        years_data[f"RAM_{year}"] = ram
                        years_data[f"Category_{year}"] = category
                
                # Add to summary dataframe
                row_data = {'SYMBOL': symbol, 'NAME OF COMPANY': name, 'INDUSTRY': industry, **years_data}
                summary_df = pd.concat([summary_df, pd.DataFrame([row_data])], ignore_index=True)
            
            # Save summary sheet
            if not summary_df.empty:
                summary_df.to_excel(writer, sheet_name='Summary', index=False)
                logger.info(f"Created summary sheet with {len(summary_df)} stocks")
        
        logger.info(f"Multi-year momentum analysis complete. Results saved to {output_file}")
        
    except Exception as e:
        logger.error(f"Error in main process: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())

if __name__ == "__main__":
    main()

2025-04-16 13:00:17,876 - INFO - Reading input file: name.xlsx
2025-04-16 13:00:18,030 - INFO - Processing 1/668: POWERGRID - Power Grid Corporation of India Limited


YF.download() has changed argument auto_adjust default to True


2025-04-16 13:00:25,387 - INFO - Successfully fetched data for POWERGRID.NS: 2538 days
2025-04-16 13:00:25,388 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:00:25,390 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:00:25,395 - INFO - Close prices has multiple columns: Index(['POWERGRID.NS'], dtype='object', name='Ticker')
2025-04-16 13:00:25,398 - INFO - Using first column of Close prices
2025-04-16 13:00:25,403 - INFO - Start price: 77.28169250488281 (<class 'numpy.float64'>), End price: 107.25914001464844 (<class 'numpy.float64'>)
2025-04-16 13:00:25,521 - INFO - Start price: 107.25914001464844 (<class 'numpy.float64'>), End price: 147.97320556640625 (<class 'numpy.float64'>)
2025-04-16 13:00:25,634 - INFO - Start price: 147.97320556640625 (<class 'numpy.float64'>), End price: 176.36973571777344 (<class 'numpy.float64'>)
2025-04-16 13:00:25,761 - INFO - Start price: 178.60760498046875 (<class 'numpy.float64'>), End price: 328.78045654296875 (<class 'numpy.float64'>

2025-04-16 13:00:46,614 - INFO - Using first column of Close prices
2025-04-16 13:00:46,615 - INFO - Start price: 67.43389892578125 (<class 'numpy.float64'>), End price: 331.693359375 (<class 'numpy.float64'>)
2025-04-16 13:00:46,754 - INFO - Start price: 331.693359375 (<class 'numpy.float64'>), End price: 418.22332763671875 (<class 'numpy.float64'>)
2025-04-16 13:00:46,903 - INFO - Start price: 418.22332763671875 (<class 'numpy.float64'>), End price: 490.7605895996094 (<class 'numpy.float64'>)
2025-04-16 13:00:47,017 - INFO - Start price: 500.37945556640625 (<class 'numpy.float64'>), End price: 766.9000244140625 (<class 'numpy.float64'>)
2025-04-16 13:00:47,669 - INFO - Processing 10/668: PRAKASH - Prakash Industries Limited
2025-04-16 13:00:48,364 - INFO - Successfully fetched data for PRAKASH.NS: 2538 days
2025-04-16 13:00:48,365 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:00:48,366 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:00:48,368 - INFO - Close prices 

2025-04-16 13:01:01,173 - INFO - Processing 17/668: PREMIER - Premier Limited
2025-04-16 13:01:02,068 - INFO - Successfully fetched data for PREMIER.NS: 2537 days
2025-04-16 13:01:02,069 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:01:02,072 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:01:02,076 - INFO - Close prices has multiple columns: Index(['PREMIER.NS'], dtype='object', name='Ticker')
2025-04-16 13:01:02,078 - INFO - Using first column of Close prices
2025-04-16 13:01:02,082 - INFO - Start price: 3.450000047683716 (<class 'numpy.float64'>), End price: 3.75 (<class 'numpy.float64'>)
2025-04-16 13:01:02,198 - INFO - Start price: 3.75 (<class 'numpy.float64'>), End price: 3.950000047683716 (<class 'numpy.float64'>)
2025-04-16 13:01:02,323 - INFO - Start price: 3.950000047683716 (<class 'numpy.float64'>), End price: 2.0999999046325684 (<class 'numpy.float64'>)
2025-04-16 13:01:02,455 - INFO - Start price: 2.049999952316284 (<class 'numpy.float64'>), End price: 

...

2025-04-16 13:42:15,319 - INFO - Start price: 2.200000047683716 (<class 'numpy.float64'>), End price: 7.900000095367432 (<class 'numpy.float64'>)
2025-04-16 13:42:16,176 - INFO - Start price: 7.900000095367432 (<class 'numpy.float64'>), End price: 14.399999618530273 (<class 'numpy.float64'>)
2025-04-16 13:42:17,039 - INFO - Start price: 14.399999618530273 (<class 'numpy.float64'>), End price: 105.05000305175781 (<class 'numpy.float64'>)
2025-04-16 13:42:17,874 - INFO - Start price: 107.1500015258789 (<class 'numpy.float64'>), End price: 156.7899932861328 (<class 'numpy.float64'>)
2025-04-16 13:42:19,360 - INFO - Processing 640/668: WSTCSTPAPR - West Coast Paper Mills Limited
2025-04-16 13:42:20,822 - INFO - Successfully fetched data for WSTCSTPAPR.NS: 2538 days
2025-04-16 13:42:20,822 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:42:20,822 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:42:20,831 - INFO - Close prices has multiple columns: Index(['WSTCSTPAPR.NS'], dt

2025-04-16 13:43:02,071 - INFO - Start price: 373.6000061035156 (<class 'numpy.float64'>), End price: 523.9000244140625 (<class 'numpy.float64'>)
2025-04-16 13:43:03,548 - INFO - Processing 648/668: YATRA - Yatra Online Limited
2025-04-16 13:43:04,602 - INFO - Successfully fetched data for YATRA.NS: 379 days
2025-04-16 13:43:04,608 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:43:04,608 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:43:04,614 - INFO - Close prices has multiple columns: Index(['YATRA.NS'], dtype='object', name='Ticker')
2025-04-16 13:43:04,616 - INFO - Using first column of Close prices
2025-04-16 13:43:04,618 - WARNING - Insufficient data for year ending 2021-09-01: 0 days
2025-04-16 13:43:05,522 - WARNING - Insufficient data for year ending 2022-09-01: 0 days
2025-04-16 13:43:06,419 - WARNING - Insufficient data for year ending 2023-09-01: 0 days
2025-04-16 13:43:07,316 - INFO - Start price: 135.89999389648438 (<class 'numpy.float64'>), End price: 

2025-04-16 13:43:46,512 - INFO - Processing 656/668: ZENITHSTL - Zenith Steel Pipes & Industries Limited
2025-04-16 13:43:47,333 - INFO - Successfully fetched data for ZENITHSTL.NS: 2537 days
2025-04-16 13:43:47,333 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:43:47,333 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:43:47,342 - INFO - Close prices has multiple columns: Index(['ZENITHSTL.NS'], dtype='object', name='Ticker')
2025-04-16 13:43:47,342 - INFO - Using first column of Close prices
2025-04-16 13:43:47,347 - INFO - Start price: 1.100000023841858 (<class 'numpy.float64'>), End price: 1.100000023841858 (<class 'numpy.float64'>)
2025-04-16 13:43:48,262 - INFO - Start price: 1.100000023841858 (<class 'numpy.float64'>), End price: 6.099999904632568 (<class 'numpy.float64'>)
2025-04-16 13:43:49,101 - INFO - Start price: 6.099999904632568 (<class 'numpy.float64'>), End price: 3.950000047683716 (<class 'numpy.float64'>)
2025-04-16 13:43:49,970 - INFO - Start price: 

2025-04-16 13:44:26,754 - INFO - Start price: 98.05000305175781 (<class 'numpy.float64'>), End price: 250.52999877929688 (<class 'numpy.float64'>)
2025-04-16 13:44:28,288 - INFO - Processing 664/668: ZOTA - Zota Health Care LImited
2025-04-16 13:44:29,565 - INFO - Successfully fetched data for ZOTA.NS: 1959 days
2025-04-16 13:44:29,565 - INFO - Detected MultiIndex DataFrame
2025-04-16 13:44:29,565 - INFO - Using 'Close' prices from MultiIndex
2025-04-16 13:44:29,579 - INFO - Close prices has multiple columns: Index(['ZOTA.NS'], dtype='object', name='Ticker')
2025-04-16 13:44:29,579 - INFO - Using first column of Close prices
2025-04-16 13:44:29,584 - INFO - Start price: 146.08056640625 (<class 'numpy.float64'>), End price: 376.24652099609375 (<class 'numpy.float64'>)
2025-04-16 13:44:30,629 - INFO - Start price: 376.24652099609375 (<class 'numpy.float64'>), End price: 273.5156555175781 (<class 'numpy.float64'>)
2025-04-16 13:44:31,690 - INFO - Start price: 273.5156555175781 (<class 'nu

KeyboardInterrupt: 